In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import math
import torch
import pickle
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import joblib
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV

# check xgboost version
from xgboost import XGBRegressor

#torch stuff
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import holidays
us_holidays = holidays.country_holidays('US')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

directory = '/content/drive/MyDrive/Thesis'
data_dir = directory + "/Data"
models_dir = directory + "/models"

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
from hyperopt.pyll.base import scope
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [4]:
df = pd.read_csv(f'{data_dir}/final_model_input_partial_scale_4.csv')

df['started_at_hourly'] = pd.to_datetime(df['started_at_hourly'])
df = df.sort_values(by=['start_station_cluster', 'started_at_hourly'])

df_test = df[pd.to_datetime(
    df['started_at_hourly']) >= pd.to_datetime("2024-01-01 00:00:00")]
df_train = df[pd.to_datetime(
    df['started_at_hourly']) < pd.to_datetime("2024-01-01 00:00:00")]

del df

In [5]:
target_scaler = joblib.load(f'{models_dir}/target_scaler.sav')

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.6.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
execfile('/content/drive/MyDrive/Thesis/models_training/model_training.py')

Using device: cpu


# Weather and Lag


In [7]:
FEATURES = ['start_station_cluster', 'started_at_year',
            'started_at_month', 'started_at_day',
            'started_at_hour',
            'started_at_week',
            'started_at_quarter',
            'started_at_dayofweek',
            'is_holiday',
            #'flag_added',
            'temp',
            'dwpt',
            'rhum',
            'prcp',
            'wdir',
            'wspd',
            'pres',
            'coco',
            "coco_lag_1_h",
            "coco_lag_2_h",
            "coco_lag_24_h",
            "demand_lag_1_h",
            "demand_lag_2_h",
            "demand_lag_24_h",
            "total_demand_1h",
            "demand_degrees_1h",
]

## XGBoost

In [8]:
model, model_preds, preds_list = train_model(df_train, df_test, FEATURES, model='xgb')

overall
MSE: 3.7203889001079857
RMSE: 1.928830967220297
MAE: 1.0242482081429387
MAPE: 0.4131493752671581
### Standard Deviation
MSE: 9.362222582871203e-16
RMSE: 2.340555645717801e-16
MAE: 2.340555645717801e-16
MAPE: 0.0
### Variance
MSE: 8.765121169122354e-31
RMSE: 5.478200730701471e-32
MAE: 5.478200730701471e-32
MAPE: 0.0

Non-zero
MSE: 7.564361687708103
RMSE: 2.750338467845022
MAE: 1.7159217272215899
MAPE: 0.35742167174646944
### Standard Deviation
MSE: 9.362222582871203e-16
RMSE: 0.0
MAE: 2.340555645717801e-16
MAPE: 5.851389114294502e-17
### Variance
MSE: 8.765121169122354e-31
RMSE: 0.0
MAE: 5.478200730701471e-32
MAPE: 3.4238754566884194e-33

#####
Zeros
MSE: 0.577435405947131
RMSE: 0.7598917067234849
MAE: 0.4587141048268745
MAPE: 0.4587141048268745
### Standard Deviation
MSE: 1.1702778228589004e-16
RMSE: 1.1702778228589004e-16
MAE: 0.0
MAPE: 0.0
### Variance
MSE: 1.3695501826753678e-32
RMSE: 1.3695501826753678e-32
MAE: 0.0
MAPE: 0.0


In [ ]:

#11-01
3.1907477810548825
1.786266436188869
0.3707247146921183

#11-09 scaled
3.6841001457201483
1.9194009861725476
0.4132425073915699

#11-09 unscaled
3.69007942678706
1.9209579450854877
0.41339645413116016

#11-28
3.6956497497043204
1.922407279871859
0.41613950241680286

0.41339645413116016

In [ ]:
suf = "_c_xgb_11_28"

model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

filename = f'{models_dir}/test_predictions_list{suf}.sav'
joblib.dump(preds_list, filename)
del preds_list

## LightGBM

In [9]:
model, model_preds, preds_list = train_model(df_train, df_test, FEATURES, model='lgbm')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.117701 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1740
[LightGBM] [Info] Number of data points in the train set: 1140300, number of used features: 25
[LightGBM] [Info] Start training from score 1.005143
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.195142 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1741
[LightGBM] [Info] Number of data points in the train set: 1140300, number of used features: 25
[LightGBM] [Info] Start training from score 1.005143
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.194146 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not e

In [ ]:
#11-15
3.8126704498726967
1.952606066228592
0.4274103374810608

#11-28
3.789268144399963
1.9466042598329951
0.4284581883842336

0.4274103374810608

In [ ]:
suf = "_c_lgbm_11_28"

model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

filename = f'{models_dir}/test_predictions_list{suf}.sav'
joblib.dump(preds_list, filename)
del preds_list

## Random Forest

In [ ]:
model, model_preds, preds_list = train_model(df_train, df_test, FEATURES, model='rf')

In [ ]:

#11-01
3.6111553268109304
1.9003040090498495
0.4143235449961506

#11-28
4.125098101884717
2.0310337520299155
0.4636683281398012



0.4143235449961506

In [ ]:
suf = "_c_rf_11_28"

model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

filename = f'{models_dir}/test_predictions_list{suf}.sav'
joblib.dump(preds_list, filename)
del preds_list